## Task
Forecast site-level inventory for each SKU for the next 30/60/90 days

#### Key insights 


In [19]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

DATA_PATH = Path().cwd().parent / "data" / "shipments.csv"
ship_df = pd.read_csv(
    DATA_PATH, parse_dates=["ship_date", "eta_date"], low_memory=False
)
pucharse_df = pd.read_csv(
    Path().cwd().parent / "data" / "purchase_orders.csv",
    parse_dates=["order_date", "promised_date"],
    low_memory=False,
)
suppliers_df = pd.read_csv(
    Path().cwd().parent / "data" / "suppliers.csv",
    low_memory=False,
)
sku_df = pd.read_csv(Path().cwd().parent / "data" / "skus.csv", low_memory=False)
transit_events_df = pd.read_csv(
    Path().cwd().parent / "data" / "transit_events.csv",
    parse_dates=["event_ts"],
    low_memory=False,
)
sites_df = pd.read_csv(Path().cwd().parent / "data" / "sites.csv", low_memory=False)
inventory_df = pd.read_csv(
    Path().cwd().parent / "data" / "inventory_snapshots.csv",
    parse_dates=["snapshot_date"],
    low_memory=False,
)

In [20]:
sites_df.head()

,site_id,region,country,site_type,operator,latitude,longitude
0,ST-00001,EMEA,Italy,Cell Site,Three,44.65748,-3.44596
1,ST-00002,APAC,South Korea,Cell Site,Singtel,19.46501,122.29595
2,ST-00003,EMEA,Austria,Warehouse,NaN,41.41491,27.75070
3,ST-00004,EMEA,Italy,Integration Center,NaN,51.17115,-1.20329
4,ST-00005,AMER,Canada,Cell Site,Verizon,-19.96994,-93.20117


In [21]:
# plot the three sites on the existing pygal world map (uses worldmap_chart and sites_df already in the notebook)
# create map if it doesn't already exist
if 'worldmap_chart' not in globals():
    import pygal
    worldmap_chart = pygal.maps.world.World()
# small manual mapping of country names -> pygal two-letter country codes
country_codes = {
    'Italy': 'it',
    'South Korea': 'kr',
    'Korea, South': 'kr',
    'Austria': 'at',
    # add more mappings if needed
}

# select the three sites
site_ids = ['ST-00001', 'ST-00002', 'ST-00003']
subset = sites_df.loc[sites_df['site_id'].isin(site_ids), 
                      ['site_id', 'region', 'country', 'site_type', 'operator', 'latitude', 'longitude']]

# add each site as a separate series so the tooltip/legend shows the site metadata
for _, r in subset.iterrows():
    code = country_codes.get(r['country'])
    label = f"{r['site_id']} | {r['region']} | {r['country']} | {r['site_type']} | {r.get('operator','') or ''} | {r['latitude']:.5f},{r['longitude']:.5f}"
    if code:
        worldmap_chart.add(label, [code])
    else:
        # fallback: add country name as series (will not position by lat/lon but will mark the country)
        worldmap_chart.add(label, [r['country']])

# render inline in the notebook
worldmap_chart.render_in_browser()  # or use .render_notebook() depending on your environment

file://C:/Users/miskibin/AppData/Local/Temp/tmpts0m276z.html


In [22]:
# merge inventory.site_id with sites.site_id 
# inventory.sku_id with sku.sku_id
df = inventory_df.merge(
    sites_df,
    how='left',
    left_on='site_id',
    right_on='site_id',
).merge(
    sku_df,
    how='left',
    left_on='sku_id',
    right_on='sku_id',
)
df

,site_id,sku_id,snapshot_date,on_hand_qty,allocated_qty,in_transit_qty,region,country,site_type,operator,latitude,longitude,vendor,category,technology,unit_weight_kg,unit_volume_m3,std_cost_usd,supplier_nominal_lead_time_days
0,ST-01489,SKU-03098,2025-10-03,9,4,2,EMEA,Germany,Integration Center,NaN,47.26070,16.73875,Samsung,Optical Transport,Dual (4G/5G),6.75,0.0779,511.52,72
1,ST-00841,SKU-00755,2025-01-08,18,2,3,EMEA,Czechia,Cell Site,Orange,43.42930,24.07693,Samsung,Cabling,4G,8.59,0.0948,462.43,79
2,ST-00771,SKU-01482,2025-04-08,7,2,4,AMER,Chile,Warehouse,NaN,32.70190,-82.41256,Nokia,Edge Router,Dual (4G/5G),23.86,0.2507,322.28,33
3,ST-01017,SKU-00406,2024-12-22,9,6,2,EMEA,Czechia,Cell Site,Three,52.09234,-3.17320,Juniper,RAN,4G,31.65,0.3327,151.63,8
4,ST-01897,SKU-02690,2025-01-31,10,4,2,EMEA,France,Cell Site,Deutsche Telekom,44.87796,-0.44155,Huawei,Optical Transport,5G,17.00,0.1717,1009.05,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,ST-01253,SKU-03013,2025-06-01,4,1,6,AMER,Canada,Integration Center,NaN,-20.86402,-63.08772,NEC,Optical Transport,4G,10.79,0.1220,280.69,74
199996,ST-01095,SKU-00358,2025-08-30,8,3,4,EMEA,France,Cell Site,Telia,59.42177,16.16755,NEC,RAN,5G,17.57,0.1904,86.21,34
199997,ST-01933,SKU-04847,2025-08-04,7,3,3,APAC,Japan,Cell Site,Telstra,31.36052,108.95056,NEC,RAN,4G,7.98,0.0883,6376.94,41
199998,ST-00909,SKU-04170,2025-03-06,9,3,1,EMEA,UK,Warehouse,NaN,59.75697,20.05305,Nokia,Antenna,Dual (4G/5G),6.00,0.0746,232.05,57


In [23]:
# 3A.1 — In-transit converted to inbound arrivals
in_transit_df = df[df["in_transit_qty"] > 0].copy()

# Estimate arrival date = snapshot_date + supplier lead time
in_transit_df["event_date"] = (
    in_transit_df["snapshot_date"]
    + pd.to_timedelta(in_transit_df["supplier_nominal_lead_time_days"], unit="D")
)

inbound_transit_events = in_transit_df[[
    "site_id",
    "sku_id",
    "event_date",
    "in_transit_qty"
]].rename(columns={"in_transit_qty": "qty_change"})

inbound_transit_events["event_type"] = "inbound_transit"


# 3A.2 — RMA inbound events
rma_returns_df = pd.read_csv(
    Path().cwd().parent / "data" / "rma_returns.csv",
    parse_dates=["rma_date"],
    low_memory=False,
)

rma_events = rma_returns_df.rename(columns={"rma_date": "event_date"}).copy()
rma_events["qty_change"] = 1      # each RMA = +1 (customize if different)
rma_events["event_type"] = "inbound_rma"

rma_events = rma_events[["site_id", "sku_id", "event_date", "qty_change", "event_type"]]
rma_events

,site_id,sku_id,event_date,qty_change,event_type
0,ST-01094,SKU-00206,2024-10-15,1,inbound_rma
1,ST-01174,SKU-02303,2023-11-05,1,inbound_rma
2,ST-01643,SKU-04015,2025-03-26,1,inbound_rma
3,ST-01539,SKU-00438,2025-02-25,1,inbound_rma
4,ST-00776,SKU-02819,2023-06-02,1,inbound_rma
...,...,...,...,...,...
39995,ST-00497,SKU-03476,2025-04-21,1,inbound_rma
39996,ST-00730,SKU-03575,2025-06-05,1,inbound_rma
39997,ST-01314,SKU-03710,2025-09-01,1,inbound_rma
39998,ST-00275,SKU-01080,2025-04-06,1,inbound_rma


In [24]:

deployments_df = pd.read_csv(
    Path().cwd().parent / "data" / "deployments.csv",
    parse_dates=["planned_date", "actual_date"],
    low_memory=False,
)

# Prefer actual_date when available
deployments_df["event_date"] = deployments_df["actual_date"].fillna(
    deployments_df["planned_date"]
)

deploy_events = deployments_df.copy()
deploy_events["qty_change"] = -1   # each deployment consumes 1 SKU
deploy_events["event_type"] = "deployment"

deploy_events = deploy_events[["site_id", "sku_id", "event_date", "qty_change", "event_type"]]
deploy_events

,site_id,sku_id,event_date,qty_change,event_type
0,ST-01502,SKU-03396,2024-02-11,-1,deployment
1,ST-01637,SKU-02125,2024-01-01,-1,deployment
2,ST-01988,SKU-00348,2025-01-01,-1,deployment
3,ST-01992,SKU-04214,2023-08-22,-1,deployment
4,ST-00840,SKU-01585,2024-04-22,-1,deployment
...,...,...,...,...,...
79995,ST-00561,SKU-03723,2025-10-01,-1,deployment
79996,ST-00243,SKU-01107,2024-05-01,-1,deployment
79997,ST-01883,SKU-03159,2024-09-21,-1,deployment
79998,ST-01409,SKU-01197,2024-01-15,-1,deployment


In [25]:
inventory_events = pd.concat(
    [
        inbound_transit_events,
        rma_events,
        deploy_events
    ],
    ignore_index=True
)

# Remove invalid/null event dates
inventory_events = inventory_events.dropna(subset=["event_date"])

# Sort for forecasting step
inventory_events = inventory_events.sort_values(["site_id", "sku_id", "event_date"]).reset_index(drop=True)

inventory_events

,site_id,sku_id,event_date,qty_change,event_type
0,ST-00001,SKU-00005,2025-05-23,2,inbound_transit
1,ST-00001,SKU-00012,2025-06-08,6,inbound_transit
2,ST-00001,SKU-00037,2025-03-15,1,inbound_rma
3,ST-00001,SKU-00063,2025-04-03,3,inbound_transit
4,ST-00001,SKU-00089,2023-11-22,-1,deployment
...,...,...,...,...,...
310122,ST-02000,SKU-04808,2025-10-21,5,inbound_transit
310123,ST-02000,SKU-04848,2023-07-12,-1,deployment
310124,ST-02000,SKU-04860,2025-06-14,3,inbound_transit
310125,ST-02000,SKU-04874,2025-06-10,3,inbound_transit


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from datetime import timedelta

# 1. Setup Snapshots & Static Data
# We only keep pairs that actually exist in your inventory snapshot
static_cols = ['site_id', 'sku_id', 'region', 'country', 'category', 'technology', 'on_hand_qty']
static_df = df[static_cols].drop_duplicates(subset=['site_id', 'sku_id']).copy()
unique_site_skus = static_df[['site_id', 'sku_id']].drop_duplicates()

# 2. Extract Demand History
demand_hist = inventory_events[inventory_events['event_type'] == 'deployment'].copy()
demand_hist['demand'] = demand_hist['qty_change'].abs()
demand_hist = demand_hist.groupby(['site_id', 'sku_id', 'event_date'], as_index=False)['demand'].sum()

# 3. Create the Time Series Grid (The Fix)
min_date = demand_hist['event_date'].min()
max_date = demand_hist['event_date'].max()
all_dates = pd.date_range(min_date, max_date, freq='D')

# Create a DataFrame just for dates
date_df = pd.DataFrame({'event_date': all_dates})

# perform a CROSS JOIN only between Valid Pairs and Dates
# This drastically reduces memory usage
ts_df = unique_site_skus.merge(date_df, how='cross')

In [ ]:


# 4. Join Demand & Fill Zeros
ts_df = ts_df.merge(demand_hist, on=['site_id', 'sku_id', 'event_date'], how='left')
ts_df['demand'] = ts_df['demand'].fillna(0)

# 5. Feature Engineering
ts_df = ts_df.sort_values(['site_id', 'sku_id', 'event_date'])

# Date Features
ts_df['month'] = ts_df['event_date'].dt.month
ts_df['weekday'] = ts_df['event_date'].dt.weekday
ts_df['day_of_year'] = ts_df['event_date'].dt.dayofyear

# Lags & Rolling Windows
g = ts_df.groupby(['site_id', 'sku_id'])

ts_df['lag_1'] = g['demand'].shift(1)
ts_df['lag_7'] = g['demand'].shift(7)
ts_df['lag_28'] = g['demand'].shift(28)

# Use transform for rolling to keep the dataframe shape
ts_df['roll_mean_7'] = g['demand'].transform(lambda x: x.rolling(7, min_periods=1).mean())
ts_df['roll_mean_28'] = g['demand'].transform(lambda x: x.rolling(28, min_periods=1).mean())

# Merge static features back
train_df = ts_df.merge(static_df.drop(columns='on_hand_qty'), on=['site_id', 'sku_id'], how='left')

# Convert Categoricals
cat_cols = ['region', 'country', 'category', 'technology', 'site_id', 'sku_id']
for c in cat_cols:
    train_df[c] = train_df[c].astype('category')

train_df = train_df.dropna()

print(f"Training Data Shape: {train_df.shape}")